In [1]:
#!pip install splinter

In [1]:
# Import dependencies
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
from selenium import webdriver

In [26]:
# Initialize Splinter Browser
with Browser('chrome') as browser:
    # Define base URL
    url = 'https://ircc.canada.ca/english/newcomers/services/index.asp#table1'
    browser.visit(url)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    # List to store all extracted data
    all_page_data = []

    # Find all rows (tr) in the table
    rows = soup.find_all('tr')

    # Iterate through each row to extract data
    for row in rows:
        # Extract Organisation Name
        org_name_element = row.find('a', string=True)  # Find the <a> tag containing the org name
        org_name = org_name_element.text.strip() if org_name_element else None

        # Extract Organisation Website
        org_website_element = row.find('a', href=True)
        org_website = org_website_element['href'] if org_website_element else None
        
        # Extract all <a> tags with href in the row
        org_address_elements = row.find_all('a', href=True)

        # Extract the second <a> tag, assuming it's the Google Maps link
        org_googlemap_element = org_address_elements[1] if len(org_address_elements) > 1 else None
        org_googlemap = org_googlemap_element['href'] if org_googlemap_element else None

        # Extract the second <a> tag in the row (if it exists)
        org_address_element = org_address_elements[1] if len(org_address_elements) > 1 else None  # Get the second <a> tag
        org_address = org_address_element.text.strip() if org_address_element else None

        # Extract Languages of Service
        lang_element = row.find('td')  # Assume languages are in a <td> tag
        if lang_element and "Languages of service:" in lang_element.text:
            org_languages = lang_element.text.split("Languages of service:")[1].strip()
        else:
            org_languages = None

        # Extract Services
        services_elements = row.find_all('li', class_="nowrap")
        org_services = [service.text.strip() for service in services_elements]
        
        # Initialize an empty list to store the texts of all <td> elements
        all_td_text = []

        # Find all <td> tags in the row (table data cells)
        td_elements = row.find_all('td')
        
        # Iterate through each <td> tag in the row
        for td in td_elements:
            # Remove all <a> tags to avoid extracting the hyperlink text
            for a_tag in td.find_all('a'):
                a_tag.decompose()  # This will remove the <a> tag and its content from the <td>
            
            # Get the text inside the <td> and strip any surrounding whitespace
            cell_text = td.get_text(strip=True)
            
            if cell_text:  # Add non-empty text to the list
                all_td_text.append(cell_text)

        # Join the list of texts from all <td> elements into a single string
        org_details = '/'.join(all_td_text)

        # Organize extracted data into a dictionary
        item_data = {
            'Organisation Name': org_name,
            'Organisation Website': org_website,
            'Organisation Address': org_address,
            'Organisation Languages': org_languages,
            'Organisation Services': org_services,
            'Organisation Google Map': org_googlemap,
            'Organisation Details': org_details  # Store the combined <td> contents
        }

        # Append the data to the list
        all_page_data.append(item_data)

    # Print or process the collected data
    for data in all_page_data:
        print(data)



{'Organisation Name': None, 'Organisation Website': None, 'Organisation Address': None, 'Organisation Languages': None, 'Organisation Services': [], 'Organisation Google Map': None, 'Organisation Details': ''}
{'Organisation Name': 'A & O - Support Services for Older Adults', 'Organisation Website': 'https://www.aosupportservices.ca/', 'Organisation Address': '200 - 207 Donald Street', 'Organisation Languages': 'English, Hindi, Punjabi, Ukrainian', 'Organisation Services': ['General settlement help', 'Help connecting with the local community', 'Services available online', 'General language training', 'Informal language learning', 'Services for seniors'], 'Organisation Google Map': 'http://maps.google.com/maps?q=200%20-%20207%20Donald%20Street%20Winnipeg%20Manitoba%20Canada', 'Organisation Details': 'Winnipeg, ManitobaR3C 1M5CanadaTelephone: 204-956-6440Languages of service: English, Hindi, Punjabi, Ukrainian/General settlement helpHelp connecting with the local communityServices availa

In [27]:
df=pd.DataFrame(all_page_data)
df

,Organisation Name,Organisation Website,Organisation Address,Organisation Languages,Organisation Services,Organisation Google Map,Organisation Details
0,None,None,None,None,[],None,
1,A & O - Support Services for Older Adults,https://www.aosupportservices.ca/,200 - 207 Donald Street,"English, Hindi, Punjabi, Ukrainian","[General settlement help, Help connecting with...",http://maps.google.com/maps?q=200%20-%20207%20...,"Winnipeg, ManitobaR3C 1M5CanadaTelephone: 204-..."
2,Accès Emploi Alberta,https://accesemploi.net/en/home/,2004 - 9499 137 Avenue NW,"English, French, Arabic, Kinyarwanda, Kirundi,...","[Get a settlement plan, Help connecting with t...",http://maps.google.com/maps?q=2004%20-%209499%...,"Edmonton, AlbertaT5E 5R8CanadaTelephone: 780-4..."
3,Accès Emploi Alberta,https://accesemploi.net/en/home/,202 - 8627 91 Street NW,"English, French, Arabic, Creole, Kinyarwanda, ...","[Get a settlement plan, Help connecting with t...",http://maps.google.com/maps?q=202%20-%208627%2...,"Edmonton, AlbertaT6C 3N1CanadaTelephone: 780-4..."
4,Accès Emploi Alberta - Online only,https://accesemploi.net/en/home/,info@accesemploi.net,"English, French, Arabic, Kinyarwanda, Kirundi,...","[Get a settlement plan, Help connecting with t...",mailto:info@accesemploi.net,"Red Deer, AlbertaTelephone: 866-490-6999Langua..."
5,Accès Emploi Alberta - Online only,https://accesemploi.net/en/home/,info@accesemploi.net,"English, French, Arabic, Kinyarwanda, Kirundi,...","[Get a settlement plan, Help connecting with t...",mailto:info@accesemploi.net,"Grande Prairie, AlbertaTelephone: 866-490-6999..."


In [24]:
# Initialize Splinter Browser
with Browser('chrome') as browser:
    # Define base URL
    url = 'https://ircc.canada.ca/english/newcomers/services/index.asp'
    browser.visit(url)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    # List to store all extracted data
    all_page_data = []

    # Find all rows (tr) in the table
    rows = soup.find_all('tr')

    # Iterate through each row to extract data
    for row in rows:
        # Extract Organisation Name
        org_name_element = row.find('a', string=True)  # Find the <a> tag containing the org name
        org_name = org_name_element.text.strip() if org_name_element else None

        # Extract Organisation Website
        org_website_element = row.find('a', href=True)
        org_website = org_website_element['href'] if org_website_element else None
        
        # Extract all <a> tags with href in the row
        org_address_elements = row.find_all('a', href=True)

        # Extract the second <a> tag, assuming it's the Google Maps link
        org_googlemap_element = org_address_elements[1] if len(org_address_elements) > 1 else None
        org_googlemap = org_googlemap_element['href'] if org_googlemap_element else None

        # Extract the second <a> tag in the row (if it exists)
        org_address_element = org_address_elements[1] if len(org_address_elements) > 1 else None  # Get the second <a> tag
        org_address = org_address_element.text.strip() if org_address_element else None

        # Extract Languages of Service
        lang_element = row.find('td')  # Assume languages are in a <td> tag
        if lang_element and "Languages of service:" in lang_element.text:
            org_languages = lang_element.text.split("Languages of service:")[1].strip()
        else:
            org_languages = None

        # Extract Services
        services_elements = row.find_all('li', class_="nowrap")
        org_services = [service.text.strip() for service in services_elements]
        
        # Initialize an empty list to store the texts of all <td> elements
        all_td_text = []

        # Find all <td> tags in the row (table data cells)
        td_elements = row.find_all('td')
        
        # Iterate through each <td> tag in the row
        for td in td_elements:
            # Remove all <a> tags to avoid extracting the hyperlink text
            for a_tag in td.find_all('a'):
                a_tag.decompose()  # This will remove the <a> tag and its content from the <td>
            
            # Get the text inside the <td> and strip any surrounding whitespace
            cell_text = td.get_text(strip=True)
            
            if cell_text:  # Add non-empty text to the list
                all_td_text.append(cell_text)

        # Join the list of texts from all <td> elements into a single string
        org_details = '/'.join(all_td_text)

        # Organize extracted data into a dictionary
        item_data = {
            'Organisation Name': org_name,
            'Organisation Website': org_website,
            'Organisation Address': org_address,
            'Organisation Languages': org_languages,
            'Organisation Services': org_services,
            'Organisation Google Map': org_googlemap,
            'Organisation Details': org_details  # Store the combined <td> contents
        }

        # Append the data to the list
        all_page_data.append(item_data)

    # Print or process the collected data
    for data in all_page_data:
        print(data)


{'Organisation Name': None, 'Organisation Website': None, 'Organisation Address': None, 'Organisation Languages': None, 'Organisation Services': [], 'Organisation Google Map': None, 'Organisation Details': ''}
{'Organisation Name': 'A & O - Support Services for Older Adults', 'Organisation Website': 'https://www.aosupportservices.ca/', 'Organisation Address': '200 - 207 Donald Street', 'Organisation Languages': 'English, Hindi, Punjabi, Ukrainian', 'Organisation Services': ['General settlement help', 'Help connecting with the local community', 'Services available online', 'General language training', 'Informal language learning', 'Services for seniors'], 'Organisation Google Map': 'http://maps.google.com/maps?q=200%20-%20207%20Donald%20Street%20Winnipeg%20Manitoba%20Canada', 'Organisation Details': 'Winnipeg, ManitobaR3C 1M5CanadaTelephone: 204-956-6440Languages of service: English, Hindi, Punjabi, Ukrainian/General settlement helpHelp connecting with the local communityServices availa

In [ ]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup

# Initialize Splinter Browser
with Browser('chrome') as browser:
    # Define the base URL (adjust the page number dynamically)
    base_url = 'https://ircc.canada.ca/english/newcomers/services/index.asp?page={}'  # Assuming the URL uses '?page=1', '?page=2', etc.

    # List to store all extracted data
    all_page_data = []

    # Loop through pages from 1 to 222
    for page_num in range(1, 223):  # Adjust the range to 1 to 222
        # Visit the page
        url = base_url.format(page_num)
        browser.visit(url)

        # Get the page content and parse it with BeautifulSoup
        html = browser.html
        soup = BeautifulSoup(html, 'html.parser')

        # Find all rows (tr) in the table
        rows = soup.find_all('tr')

        # Iterate through each row to extract data
        for row in rows:
            # Extract Organisation Name
            org_name_element = row.find('a', string=True)  # Find the <a> tag containing the org name
            org_name = org_name_element.text.strip() if org_name_element else None

            # Extract Organisation Website
            org_website_element = row.find('a', href=True)
            org_website = org_website_element['href'] if org_website_element else None

            # Extract all <a> tags with href in the row
            org_address_elements = row.find_all('a', href=True)

            # Extract the second <a> tag, assuming it's the Google Maps link
            org_googlemap_element = org_address_elements[1] if len(org_address_elements) > 1 else None

            # Extract the href (Google Maps URL) from the second <a> tag
            org_googlemap = org_googlemap_element['href'] if org_googlemap_element else None

            # Extract the second <a> tag in the row (if it exists)
            org_address_element = org_address_elements[1] if len(org_address_elements) > 1 else None  # Get the second <a> tag

            # Extract the text of the second <a> tag (address) if it exists
            org_address = org_address_element.text.strip() if org_address_element else None

            # Extract Languages of Service
            lang_element = row.find('td')  # Assume languages are in a <td> tag
            if lang_element and "Languages of service:" in lang_element.text:
                org_languages = lang_element.text.split("Languages of service:")[1].strip()
            else:
                org_languages = None

            # Extract Services
            services_elements = row.find_all('li', class_="nowrap")
            org_services = [service.text.strip() for service in services_elements]

            # Extract text content from all <td> elements (excluding links)
            all_td_text = []
            td_elements = row.find_all('td')
            for td in td_elements:
                for a_tag in td.find_all('a'):
                    a_tag.decompose()  # Remove <a> tags from the text
                all_td_text.append(td.get_text(strip=True))

            # Join the list of <td> texts with a "/" separator
            org_details = '/'.join(all_td_text)

            # Organize extracted data into a dictionary
            item_data = {
                'Organisation Name': org_name,
                'Organisation Website': org_website,
                'Organisation Address': org_address,
                'Organisation Languages': org_languages,
                'Organisation Services': org_services,
                'Organisation Google Map': org_googlemap,
                'Organisation Details': org_details
            }

            # Append the data to the list
            all_page_data.append(item_data)

    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(all_
